In [1]:
# Import Dependencies
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

## Wine Quality

## Description
The combined dataset includes red and white variants of the Portuguese "Vinho Verde" wine. Only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

#### Attribute Information:

Continuous input variables (based on physicochemical tests):
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulphates
- alcohol

Labeled input variables (based on sensory and category):
- quality (score between 0 and 10)
- color (White: 0; Red: 1)



## Source
https://archive.ics.uci.edu/ml/datasets/wine+quality

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read the csv file into a pandas DataFrame called `wine_df`

wine_df = pd.read_csv('wine (4).csv')  
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0


## Use K-Means to predict clusters

## Find the best value for _k_ using an Elbow Curve

In [4]:
# Setup for loop

inertia = []
k = list(range(1, 11))

In [5]:
# Look for the best k

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(wine_df)
    inertia.append(km.inertia_)

In [6]:
# Plot the Elbow Curve using plotly

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

px.scatter(x=k,y=inertia,labels={"x":"Number of clusters","y":"Inertia"}) 

In [7]:
# Initialize a model with K based on the Elbow plot above

model = KMeans(n_clusters=3, random_state=5)

In [8]:
# Fit the model

model.fit(wine_df)

KMeans(n_clusters=3, random_state=5)

In [9]:
# Add a new `class` column to wine_df

wine_df["class"] = model.labels_
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color,class
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0,2
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0,2


In [10]:
# Create a 3D scatter plot to visualize our clustering 
# usings `total sulfur dioxide` and any two other features

px.scatter_3d(wine_df,x="total sulfur dioxide",y="fixed acidity",z="volatile acidity",color="class")


Notice how `total sulfur dioxide` basically drives the decision of `class`. Why might this be? Do you think the scale of values compared to all other features matters? Confirm this hypothesis by next performing scaling.  

## Standard Scalar

Documentation: [StandarScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [11]:
# Drop the class from our wine data

wine_df = wine_df.drop("class",axis=1) 

In [12]:
# Scale all fields with the StandardScaler() function based on the documentation above.
# Initialize a `StandardScaler` object

scaler = StandardScaler()

In [13]:
# Fit and Transform the `wine_df` with the StandardScaler 
# Store the results in the variable `scaled_features`

scaled_features = scaler.fit_transform(wine_df)

In [14]:
# Creates a new DataFrame based on the `scaled_featuers` and the original index and column values
wine_df_scaled = pd.DataFrame(scaled_features, index=wine_df.index, columns=wine_df.columns)
wine_df_scaled.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558,0.207999,-0.571367
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615,0.207999,-0.571367
2,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521,0.207999,-0.571367
3,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,0.207999,-0.571367
4,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,0.207999,-0.571367


## Find a new best value for _k_ of scaled data using an Elbow Curve

In [15]:
# Setup for loop

inertia = []
k = list(range(1, 11))

In [16]:
# Look for the new best k

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(wine_df_scaled)
    inertia.append(km.inertia_)

In [17]:
# Plot the Elbow Curve using plotly

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

px.scatter(x=k,y=inertia,labels={"x":"Number of clusters","y":"Inertia"})

In [18]:
# Initialize a mew model with K based on the Elbow plot above

model1 = KMeans(n_clusters=4, random_state=5)

In [19]:
# Fit the new model

model1.fit(wine_df_scaled)

KMeans(n_clusters=4, random_state=5)

In [20]:
# Add a new class column to wine_df_scaled

wine_df_scaled["class"] = model1.labels_
wine_df_scaled.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color,class
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558,0.207999,-0.571367,2
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615,0.207999,-0.571367,0
2,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521,0.207999,-0.571367,0
3,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,0.207999,-0.571367,2
4,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,0.207999,-0.571367,2


In [21]:
wine_df_scaled["class"].value_counts()

0    2928
2    1952
3     987
1     630
Name: class, dtype: int64

In [22]:
# Create a 3D scatter plot to visualize our clustering 
# usings the same features as before

px.scatter_3d(wine_df_scaled,x="total sulfur dioxide",y="fixed acidity",z="volatile acidity",color="class")


Does `total sulfur dioxide` still drive the decision of `class` or do new features have an impact?

Other new features have an impact because based on the 3-D graph we can visibly see our points mix. Whereas before we can see the clusters were seperated neatly.